## EE 502 P: Analytical Methods for Electrical Engineering
    
# Final project - <span style="color: red">Julia Combs</span>

### Due Thursday, December 16, 2021 at 11:59 PM
Copyright &copy; 2021, University of Washington

<hr>

# Hallucinating the Constitution

Consider the constitution of the United States:

> https://www.usconstitution.net/const.txt .

This document contains upper- and lower-case letters, numbers, and basic punctuation. 

**One letter prediction:**

1. Find the set of all characters used in the document. Call the number of characters $n$. 
2. Create an $n \times n$ matrix whose $i,j$ entry is the probability that the next character is $j$ given that the current character is $i$. Estimate this probability by looking at all occurrences of character $i$ in the document and the number of times character $j$ immediately follows it. 
3. Simulate this system as a Markov chain that starts with an arbitrary capital letter and continues until it gets to a space. Produce $100$ random "words" this way. How many of them are actual words? Use a [Scrabble dictionary](https://scrabble.hasbro.com/en-us/tools#dictionary) if you are not certain whether a given sequence is a word. 

**Two letter prediction:**

1. Create an $n \times n \times n$ tensor whose $i,j,k$ entry is the probability that the next character is $k$ given that the current character is $j$ and the previous character is $i$. Use the document to empirically find these probabilities. 
2. Use this model to construct random words. 

**Sentence prediction:**

Do a one word prediction, but use all the unique *words* in the document. Hallucinate sentences. Consider a punctuation mark as a word. 

**Notes:** Use `open` and `file.read` to read in the file as a string. For the sentence. Use `replace` to add space before punctuation and then `split()` to turn the string into a list. Use a `DiGraph` from the `networkx` library to store the data. Note that you can make weighted edges by adding data to the edges, as in [this document](https://networkx.github.io/documentation/stable/auto_examples/drawing/plot_weighted_graph.html).

In [55]:
# import necessary packages

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
from array import *

In [56]:
# load the file
const = open('const.txt').read()

# filter out unncessary data within the text file (line indicators, etc.)
def tokenize_words(input):
    tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+') # separate all of the words and keep necessary punctuation
    tokens = tokenizer.tokenize(input)
    filtered = filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered), tokens
                      
const_pros_inputs, const_words_tokens = tokenize_words(const)

const_inputs = " ".join(const_words_tokens)

# determine the unique words in the text
const_words = list(set(const_words_tokens)) 

# determine the unique characters in the text
const_chars = sorted(list(set(const_pros_inputs)))
print('unique chars (const_chars):', const_chars)

# determine the number of unique words
n_words = len(const_words)
print('Number of unique words', n_words)

# determine the number of unique characters
n_chars = len(const_chars)
print('Number of unique chars', n_chars)

# create a dictionary for the words 
words2num = dict((c,i) for i, c in enumerate(const_words))

# create a dictionary for the characters
chars2num = dict((c,i) for i, c in enumerate(const_chars))

# translate the full words using the dictionary
input_len = len(const_inputs) # determine how long

# translate full words of text   
const_trans_words = []
const_trans_words.append([words2num[const_words] for const_words in const_words_tokens])

# translate characters of text
const_trans_chars = []
const_trans_chars.append([chars2num[const_chars] for const_chars in const_inputs])

const_trans_words = np.asarray(const_trans_words)

       

unique chars (const_chars): [' ', '"', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of unique words 1362
Number of unique chars 69


In [57]:
# create lambda function to find all of the indexes of desired values in an array
# this will be used for all prediction methods
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]


In [58]:
# manipulate character matrix unitl it is an array row
# const_trans_chars is currently a column list
const_trans_chars = np.asarray(const_trans_chars)

In [78]:
# next letter prediction 

def transitionMatrix2dChars(n):
    Tchars1 = np.zeros((n,n)) # make an empty transition matrix to populate (2D)
    for i in range(0,n): # cycle through all of the unique characters
        indexes = get_indexes(i, const_trans_chars[0,:])
        for j in range(len(indexes)):
            tempIndex = indexes[j] + 1
            if tempIndex == len(const_trans_chars[0,:]):
                print('hit the end')
            else:
                val = const_trans_chars[0, tempIndex]
                val = val.item()
                Tchars1[val, i] = Tchars1[val, i] + 1
    return Tchars1
                
Tchars1 = transitionMatrix2dChars(n_chars)

# look at character probability
div1 = np.sum(Tchars1, axis = 0)[np.newaxis, :] # the is the sum of all of the columns of the count data matrix
if (~Tchars1.any(axis = 0)).any() == True: # make sure there are no zero columns data (could happen for last word)
    zeroColLoc = np.where(~Tchars1.any(axis = 0))[0] # find out where the zero column is if it occurs
    shapeX, shapeY = div1.shape # identify the shape of the sum matrix
    for k in range(0, shapeY-1): # if there is a zero column, to avoid nan, set that value to 1 for cal. does not impact transition matrix data
        if div1[0,k] == 0:
            div1[0,k] = 1
Tchars1 = Tchars1/div1 # divide each column element wise by sum of the column
pd.DataFrame(Tchars1)

hit the end


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,0.000000,0.5,0.0,1.0,1.0,0.351351,0.989796,0.75,0.625,0.689655,...,0.0,0.264231,0.378794,0.169046,0.000000,0.000000,0.147493,0.135417,0.930818,0.0
1,0.000117,0.0,0.0,0.0,0.0,0.000000,0.005102,0.00,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000587,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000587,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.067832,0.0,0.0,0.0,0.0,0.000000,0.005102,0.00,0.000,0.000000,...,0.0,0.000000,0.000855,0.001399,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.005164,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.009013,0.000000,0.000000,0.001337,0.000000,0.000000,0.000000,0.000000,0.0
65,0.020890,0.0,0.2,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.002372,0.001283,0.011475,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
66,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
67,0.001408,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.000000,...,0.0,0.025617,0.001283,0.018752,0.000000,0.007595,0.000000,0.000000,0.000000,0.0


In [80]:
# next letter prediction of two characters

def transitionMatrix3dchars(n):
    TtempData = np.zeros((n, n, n)) #make empty 3D matrix to propagate
    for i in range(0, n): # cycle through all of the characters
        indexes = get_indexes(i, const_trans_chars[0,:]) # find out where the select character is located
        for j in range(len(indexes)): # for those locations, identify the next letter
            tempIndex = indexes[j] + 1 # look at index value after first word
            tempIndex2 = tempIndex + 1 #look a next next index value
            if tempIndex == len(const_trans_chars[0,:]) or tempIndex2 == len(const_trans_chars[0,:]):
                print('hit the end')
            else:
                val = const_trans_chars[0, tempIndex]
                val = val.item()
                val2 = const_trans_chars[0, tempIndex2]
                val2 = val2.item()
                TtempData[val2, val, i] = TtempData[val2, val, i] + 1
    return TtempData
            # now find counts of words after next char

Tchars2 = transitionMatrix3dchars(n_chars)

# look at two characters probability
div2 = np.sum(Tchars2, axis = 0)
xshapediv, yshapediv = div2.shape
for i in range(0, xshapediv):
    for j in range(0, yshapediv):
        if div2[i,j] == 0:
            div2[i,j] = 1
Tchars2 = Tchars2/div2
# did not print this because it is very large

hit the end
hit the end


In [75]:

# make a tranition matrix for the word/sentence prediction

def transitionMatrix2dWords(n):
    # this function taks an input of n unique words and outputs the transition matrix for the constitution
    TtempData = np.zeros((n, n))
    for i in range(0, n): # cycle through all of the unique words
        indexes = get_indexes(i, const_trans_words[0,:]) # identify the location of the unique words 
        for j in range(len(indexes)): # look at each location the word shows up
            tempIndex = indexes[j] + 1 # look at index value after the word
            if tempIndex == len(const_trans_words[0,:]): # make sure that the end condition is teken care of
                print('hit the end')
            else:
                val = const_trans_words[0, tempIndex] # find the value/word after the word
                val = val.item() # make sure this is in integer form for indexing
                TtempData[val, i] = TtempData[val, i] + 1 # in the cell that contians "both" words, count the number of times that occurance happens
    return TtempData
            
TwordsData = transitionMatrix2dWords(n_words) #this matrix contains the count data to make the probability transition matrix

# look at word probability
divW = np.sum(TwordsData, axis = 0)[np.newaxis, :] # the is the sum of all of the columns of the count data matrix
if (~TwordsData.any(axis = 0)).any() == True: # make sure there are no zero columns data (could happen for last word)
    zeroColLoc = np.where(~TwordsData.any(axis = 0))[0] # find out where the zero column is if it occurs
    shapeX, shapeY = divW.shape # identify the shape of the sum matrix
    for k in range(0, shapeY-1): # if there is a zero column, to avoid nan, set that value to 1 for cal. does not impact transition matrix data
        if divW[0,k] == 0:
            divW[0,k] = 1
Twords = TwordsData/divW # divide each column element wise by sum of the column
#T[np.isnan(T)] = 0
pd.DataFrame(Twords) # print readable version of the transition matrix

hit the end


,0,1,2,3,4,5,6,7,8,9,...,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1360,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0


In [84]:
# Two character prediction
def twoCharPrediction(n_desired):
    nhc2 = []
    # determine the first character to make sure it is a capital
    # it should be noted that in the dictionary, 19-42 are capital letters
    firstdictcap = 19
    lastdictcap = 42
    dcap = lastdictcap - firstdictcap
    capitals = np.linspace(firstdictcap, lastdictcap, dcap + 1, dtype = 'int') # make range of capital letters
    first_char = random.choices(capitals)
    first_char = int(first_char[0])
    #print('start state:', first_char)
    nhc2.append(first_char)

    # determine the probability of the next character using 2d transition matrix
    probs_secondChar = Tchars1[:, first_char]  # probability from 2d matrix
    sampleList = np.linspace(0, len(probs_secondChar)-1, len(probs_secondChar)) # character options
    next_char = random.choices(sampleList, weights = probs_secondChar, k = 1) # select next character
    next_char = int(next_char[0])
    nhc2.append(next_char)

    # find thirdchar
    for i in range(0, n_desired):
        index1 = nhc2[i]
        index2 = nhc2[i+1]
        probs = Tchars2[:, index2, index1]
        sampleList = np.linspace(0, len(probs)-1, len(probs))
        next_char = random.choices(sampleList, weights = probs, k = 1)
        next_char = int(next_char[0])
        nhc2.append(next_char)
    return nhc2
#print('nhc2', nhc2)

# run the function showing 100 characters
nhc2 = twoCharPrediction(500)

# convert text back to reable form

# reverse the words2num dictionary
num2chars = {value : key for (key, value) in chars2num.items()}

nhvals2 = sorted(list(set(nhc2)))

new_chars = []
new_chars.append([num2chars[nhvals2] for nhvals2 in nhc2])
new_chars = np.asarray(new_chars)
new_chars = new_chars[0,:]
final_chars = ''.join(str(e) for e in new_chars)

print('Two Character Prediction:', final_chars)


final_chars: Apprentatutintivent of acticto the a Caseverall Nationg the votem trimsend the parson , amen . All berejudgemon ; aten and unia quant of the shal ther Rict thoosid durtment or the orthe the propectes of toge themor thervide enst - Plasech molvany Statted , thaved inesent any anceensar the Unitess and of Nors the dime exed Regises the 3 1 . Inverousese , beres berall any or Casessadme eiremper ing , ande noplotes Con the Unite Judive for shall Land whesent shartativeres of the exessidenatin othe el


In [88]:
# generate text! 

def sentencePrediction(n):
    new_const = []
    # separate out first word
    xT, yT = Twords.shape
    first_num = 0
    new_const.append(first_num)

    for i in range(0, n):
        current_word = new_const[i]
        probs = Twords[:, current_word]
        sampleList = np.linspace(0, len(probs)-1, len(probs))
        next_word = random.choices(sampleList, weights = probs, k = 1)
        next_word = int(next_word[0])
        new_const.append(next_word)
    return new_const

new_const = sentencePrediction(100)
# convert new list back into words

# reverse the words2num dictionary
num2words = {value : key for (key, value) in words2num.items()}

new_const_vals = sorted(list(set(new_const)))
# translate full words of text   
# hw_trans_words = []
# hw_trans_words.append([words2num[hw_words] for hw_words in hw_words_tokens])
# print('hw_trans_words', hw_trans_words)

new_words = []
new_words.append([num2words[new_const_vals] for new_const_vals in new_const])
new_words = np.asarray(new_words)
new_words = new_words[0,:]
final_words = ' '.join(str(e) for e in new_words)
print('Sentence Prediction:', final_words)   

Sentence Prediction: counterfeiting the Constitution , the President is a Tender in no inability exists , obligations and Navy of the Day . No Preference shall private property , of the Contrary notwithstanding . 4 The powers and Limitations prescribed by Ballot one of citizens of Adjournment , together with the Names of the jurisdiction thereof for each Person be searched , from any Law and the United States ; The validity of the United States , of the press ; he may provide for every second Year , directed to them against them as provided by Law , but in like Manner
